# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
from prettytable import PrettyTable

import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

167


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set Keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# filename
file = 'event_datafile_new.csv'

In [9]:
def insert_data(query, session, values, dtypes, file):
    """
    Pipeline to insert data to a Apache Casssandra table from CSV file    
    Args:
        query (str): INSERT statement query
        session (object): Apache Casssandra session
        values (list): list of indicies to pass to INSERT statement
        dtypes (list): list of types to convert `values`
        file (str): CSV file path   
    Returns:
        None
    """
    assert(len(values) ==  len(dtypes))
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(query, tuple([dtype(line[x]) for x, dtype in zip(values, dtypes)]))

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


In [10]:
# Create table in order to answer Question #1
query = """CREATE TABLE IF NOT EXISTS artist_song_length
(sessionId int,
itemInSession int,
artist text,
song text,
length float,
PRIMARY KEY (sessionId, itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [11]:
# query
query = "INSERT INTO artist_song_length (sessionId, itemInSession, artist, song, length)"
query = query + " VALUES (%s, %s, %s, %s, %s)"
# values and data types
values = [8, 3, 0, 9, 5]
dtypes = [int, int, str, str, float]
# insert to table
insert_data(query, session, values, dtypes, file)

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
# Perform SELECT statement to verify that the data has been inserted into table
query = """SELECT artist, song, length
From artist_song_length
WHERE sessionId = 338 and itemInSession = 4;"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

In [13]:
# use pretty table to display data in tabular form and include headings
t = PrettyTable(['Artist', 'Song', 'Length'])
for row in rows:
    t.add_row([row.artist, row.song, row.length])
print(t)

+--------+------+--------+
| Artist | Song | Length |
+--------+------+--------+
+--------+------+--------+


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [14]:
# Create table in order to answer Question #2
query = """CREATE TABLE IF NOT EXISTS artist_song_user
(sessionId int,
userId int,
itemInSession int,
artist text,
song text,
firstName text,
lastName text,
PRIMARY KEY ((sessionId, userId), itemInSession));"""
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [15]:
# query
query = "INSERT INTO artist_song_user (sessionId, userId, itemInSession, artist, song, firstName, lastName)"
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
# values and data types
values = [8, 10, 3, 0, 9, 1, 4]
dtypes = [int, int, int, str, str, str, str]
# insert to table
insert_data(query, session, values, dtypes, file)

In [16]:
# Perform SELECT statement to verify that the data has been inserted into table
query = """SELECT artist, song, firstName, lastName
From artist_song_user
WHERE sessionId = 182 and userId = 10
ORDER BY itemInSession;"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)  
    
# use pretty table to display data in tabular form and include headings
t = PrettyTable(['Artist', 'Song', 'First Name', 'Last Name'])
for row in rows:
    t.add_row([row.artist, row.song, row.firstname, row.lastname])
print(t)

+-------------------+------------------------------------------------------+------------+-----------+
|       Artist      |                         Song                         | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [17]:
# Create table in order to answer Question #3
query = """CREATE TABLE IF NOT EXISTS user_song
(song text,
userId int,
firstName text,
lastName text,
PRIMARY KEY (song, userId));"""
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [18]:
# query
query = "INSERT INTO user_song (song, userId, firstName, lastName)"
query = query + " VALUES (%s, %s, %s, %s)"
# values and data types
values = [9, 10, 1, 4]
dtypes = [str, int, str, str]
# insert to table
insert_data(query, session, values, dtypes, file)

In [19]:
# Perform SELECT statement to verify that the data has been inserted into table
query = """SELECT firstName, lastName
From user_song
WHERE song = 'All Hands Against His Own';"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e) 
    
# use pretty table to display data in tabular form and include headings
t = PrettyTable(['First Name', 'Last Name'])
for row in rows:
    t.add_row([row.firstname, row.lastname])
print(t)

+------------+-----------+
| First Name | Last Name |
+------------+-----------+
+------------+-----------+


### Drop the tables before closing out the sessions

In [20]:
query = """DROP TABLE IF EXISTS artist_song_length;"""
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [21]:
query = """DROP TABLE IF EXISTS artist_song_user;"""
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [22]:
query = """DROP TABLE IF EXISTS user_song;"""
try:
    session.execute(query)
except Exception as e:
    print(e) 

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()